In [83]:
%%capture

!pip install bs4
!pip install nbformat
!pip install pandas
!pip install geopandas
!pip install requests
!pip install --upgrade xlrd
!pip install openpyxl

In [84]:
%%capture

%run ./Tidy.ipynb

# Phase de modélisation: tentative de prédiction de la consommation et de la légalisation

Dans cette seconde phase du projet, nous allons nous appuyer sur les statistiques descriptives produites juste avant afin d'orienter nos choix de modélisation.

Nous allons ici présenter les deux approches que nous avons prises. En effet, nous avions d'abord en tête avec ce projet d'essayer de prédire la légalisation ou non du cannabis dans un pays donné à partir des données mises à disposition par l'EUDA. Cette première approche s'est avérée infructueuse après de nombreux essais; et nous expliquerons les raisons de cet échec et les conclusions que nous en avons tiré dans une partie "bonus", à la fin de ce notebook. En effet, même si le modèle obtenu n'était pas performant, il nous a tout de même permis de tirer des conclusions intéressantes sur notre jeu de données.

La principale raison pour laquelle ce premier modèle s'est avéré infructueux tenait notamment au manque de données disponibles, et au nombre restreint d'observations à notre disposition. Nous avons alors mis en place plusieurs solutions à ce problème:

- En premier lieu, nous avons changé la variable à prédire. Plutôt que de se concentrer sur la législation, trop peu corrélée aux données disponibles, nous nous attacherons plutôt à tenter d'estimer la part de la consommation de cannabis dans chaque pays, en utilisant des variables socioéconomiques. Cela exige alors de changer de modèle, en abandonnant le classifieur initialement prévu à différents modèles de régression.

- Ensuite, nous avons tenu à rassembler des données plus conséquentes sur chaque pays, afin d'entraîner plus finement chaque modèle. On a ainsi pu récupérer en complément des données sur la consommation de drogues, d'autres données sur les admissions à l'hôpital liées à la consommation de drogues, sur le nombre d'infractions commises liées au trafic, ou encore des données macroéconomiques simples sur chaque pays (PIB, IDH, tau de criminalité...). On espère ainsi pouvoir entraîner un modèle plus performant

## 1. Phase de préparation des données

Comme à chaque fois que l'on souhaite entraîner un modèle, **il convient avant tout de préparer et nettoyer les données au préalable.** Nous effectuerons cette tâche selon le schéma suivant:

1. Tout d'abord la collecte et la restriction, dans la masse de données mises à notre disposition, des seules variables qui nous intéressent réellement afin d'effectuer notre entraînement, en ignorant les variables moins intéressantes afin d'obtenir un *dataframe* clair et lisible, ne contenant aucun **NaN** ou valeur nulle.

2. Ensuite la préparation de ces données, notamment la standardisation et la normalisation de celles-ci afin de pouvoir effectuer notre régression correctement.

### 1.1 Nettoyage et restriction des données

#### 1.1.1 Une fonction d'agrégation bien utile

Le premier problème à régler est le suivant: dans notre jeu de données nous disposons de beaucoup de variables identiques, relevées année par année. Or il se peut dans beaucoup de cas que l'on dispose de données récentes pour certains pays, mais pas pour d'autres pour lesquels il faut remonter quelques années en arrière. Cela a pour conséquence un jeu de donnée très troué, avec beaucoup de **NaN** dont on aimerait se débarrasser.

Afin de limiter au maximum le nombre de données vides, **on se décide alors à ne garder pour chaque pays que les données les plus récentes disponibles**. Cela nous permet de minimiser le nombre de valeurs manquantes puisqu'au lieu de ne considérer que l'année ou il y a le moins de trous, on écrase toute les années en ne gardant que la donnée la plus récente (ce qui soit dit en passant a tout de même un sens; utiliser le chiffre le plus récent semble plus raisonnable que de choisir arbitrairement l'année pour laquelle on a le plus de données)

Cela est implémenté par une simple fonction, qui fonctionne sur notre structure de donnée définie au préalable (pour plus d'informations, se référer à la deuxième partie du Notebook `Tidy.ipynb`)

In [85]:
def get_latest_data(df, new_df, variable):
    """ Parcours la table pour créer une nouvelle variable qui prend en compte les données les plus récentes disponibles
        Ce afin de pallier aux fait qu'on manque de données récentes pour certaines variables"""

    columns = list(df.columns.values)
    good_col = []

    new_df['Country'] = df['Country']

    for i in range(len(columns)):
        if variable in columns[i]:
            good_col.append(columns[i])

    for index in df.index:
        for column in good_col:
            if pd.isna(df.at[index, column]) == False:
                new_df.at[index, variable] = df.at[index, column]
                break

    new_df.dropna(inplace = True)

    return new_df

## 3. BONUS: Une première idée rejetée

Nous avons en premier lieu essayé de prédire la légalisation ou non du cannabis au sein d'un même pays à l'aide d'un SVM prenant en entrée la prévalence dans chaque pays.

Cette première approche n'a pas porté les fruits escomptés et ce pour plusieurs raisons, la principale étant une inadéquation entre notre jeu de données (trop faible) et le modèle choisi. Cette dernière section nous permet d'illustrer les raisons de cet échec et de tirer des conclusions sur la démarche à adopter lors du choix et de la mise en place de modèles.

### 3.1 Premier entraînement du classifieur sur des données de prévalence

On s'était d'abord restreint dans le cadre de la prédiction de la légalisation ou non du cannabis dans chaque pays, au pourcentage d'individus ayant déjà consommé du cannabis dans leur vie pour chaque pays.
On se rend alors vite compte que cette restriction nous donne un jeu de données très faible (1 observation par pays, soit seulement 29 observations). Ce qui va poser problème par la suite.

In [86]:
import pandas as pd

# On récupère la table de légalisation que l'on a nettoyé en amont

legal = legal_pays_clean

# On prend la prévalence sur la vie entière

df = pd.read_csv('https://raw.githubusercontent.com/refouch/Projet_Python_2A/refs/heads/main/Data/EUDA/edr2024-gps-current-table-1.csv')

dfcurrent = df[df['Substance'] == 'Cannabis']

dfcurrentadults = dfcurrent[dfcurrent['Age'] == 'All adults (15-64)']

dfcurrentadultslf = dfcurrentadults[dfcurrentadults['Recall period'] == 'Lifetime']

# On effectue un merge les deux tables
dfadultslf_legal = dfcurrentadultslf.join(legal, on = 'Country')

On a ensuite mis en place notre classifieur de manière classique, en utilisant les fonctions déjà fournies par ScikitLearn. Quelques remarques cependant sur la construction de notre modèle:

- On choisit de recoder les variables catégorielles sur la légalité en une variable binaire: 0 si le cannabis est Illegal, 1 si il est Légalisé ou Décriminalisé. Après plusieurs test en effet, nous avons conclu qu'il était préférable de se limiter à une variable binaire et de ne pas marquer la spécificité de la décriminalisation, notre jeu de données étant déjà suffisamment restreint.

- On choisit de diviser nos données de manière assez classique, en gardant un échantillon de 20% pour tester notre modèle tout en gardant 80% pour l'entraînant. Cela pose déjà un problème puisque cela ne laisse en soi que 6 observations sur lesquelles tester notre modèle, et 22 sur lesquelles l'entraîner. Ce qui amènera par la suite de fortes disparités d'une phase d'entraînement à l'autre.

- Pour notre premier essai, nous tentons simplement de fournir à notre modèle la prévalence sur la vie entière (i.e le pourcentage d'individus par pays ayant déjà fumé du cannabis dans sa vie), ainsi que la prévalence en fonction du sexe. Cela s'avèrera évidemment insuffisant, mais cela nous permettra par la suite d'illustrer au mieux les problèmes rencontrés et d'appuyer les conclusions que nous en avons tiré. Ces données on en outre le bon goût d'être de simples pourcentages, ce qui ne permet de ne pas avoir à les normaliser au préalable, ni d'avoir a se soucier d'éventuels outliers.


Ceci étant dit, nous pouvons passer à la phase d'entraînement du modèle. **Nous allons ainsi l'entraîner deux fois en prenant à chaque fois une *seed* différente** ce qui va nous permettre de démontrer l'inconstance de notre modèle, et finalement sa faible utilité.

In [ ]:
from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import warnings

warnings.filterwarnings('ignore')

# Recodage des variables catégorielles et division de notre échantillon

dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})
dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})

X = dfadultslf_legal[['Prevalence (%)','Males (%)','Females (%)']]
Y = dfadultslf_legal['Recreational_num']

# Création de deux échantillonages différents, selon une seed prédéterminée
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=1241)
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X,Y, test_size=0.2, random_state=76835790)

# Entraînement du classifieur sur nos deux échantillonages
svm_classifier = svm.SVC(C=1, kernel='linear')

svm_classifier.fit(X_train, Y_train)
Y_pred = svm_classifier.predict(X_test)

svm_classifier.fit(X2_train, Y2_train)
Y2_pred = svm_classifier.predict(X2_test)

# Accuracy score 
accuracy_score = sklearn.metrics.accuracy_score(Y_test, Y_pred)

accuracy_score_2 = sklearn.metrics.accuracy_score(Y2_test, Y2_pred)

print('Précision du modèle après entraînement sur le premier découpage: ', accuracy_score)
print('Précision du modèle après entraînement sur le second découpage: ', accuracy_score_2)

Précision du modèle après entraînement sur le premier découpage:  0.16666666666666666
Précision du modèle après entraînement sur le second découpage:  0.8333333333333334


On remarque donc qu'en fonction de notre découpage, la précision du modèle peut s'avérer une fois très bonne, et une autre fois absolument médiocre. De cela on ne peut conclure qu'une seule chose: notre modèle prédit essentiellement au hasard (ici les deux seed ont été bien choisies pour illustrer notre démarche, mais dans la plupart des test réalisés on se retrouvait avec une *accuracy* oscillant autour de 0.5, soit l'espérance de prédire correctement une variable binaire au hasard)

Ainsi, la variable utilisée pour l'entraîner n'a qu'un pouvoir explicatif très faible sur celle que l'on cherche à prédire.
Autrement dit: la prévalence de la consommation du cannabis dans chaque pays n'est que très faiblement (voire pas du tout) corrélée à l'état de la législation dans ce pays. On pouvait en réalité déjà s'en rendre compte lors de la phase de statistiques descriptives: si la consommation de cannabis est illégale en France comme en Turquie, ces deux pays constituent les valeurs extrêmes de notre échantillon quand à la prévalence sur la vie entière (56% contre 2%)

Il devient alors évident **qu'il nous faut plus de données, et des données plus pertinentes afin d'établir un classifieur plus performant**. C'est ce que nous avons tenté de faire par la suite, et ce qui nous a posé encore d'autres problèmes, desquels nous pouvons tirer encore d'autres conclusions intéréssantes.

### 3.2 Second entraînement du classifieur, cette fois sur des données judiciaires

Notre principal problème étant le manque de corrélation entre nos variables explicatives et notre variable à prédire, nous sommes allé chercher des données de l'EUDA qui pourrait le mieux expliquer la légalisation ou non du cannabis dans un pays donné.

On s'est alors arrêté sur les données judiciaires fournies par l'agence, à savoir le nombre d'infractions commises dans chaque pays concernant la consommation, la vente, ou encore la production de cannabis.
On imagine a priori que ces nouvelles données nous permettrons de mieux prédire la légalisation ou non du cannabis dans un pays donné.


In [88]:
import pandas as pd

# On récupère la table de légalisation que l'on a nettoyé en amont

legal = legal_pays_clean

# On prend cette fois les données judiciaires, qu'on passe par notre fonction de tri définie en amont

dfcurrent = table_infractions_legales

df_infrac = pd.DataFrame()

df_infrac = get_latest_data(dfcurrent,df_infrac,'cannabis.conso_x')
df_infrac = get_latest_data(dfcurrent,df_infrac,'cannabis.conso_y')
df_infrac = get_latest_data(dfcurrent,df_infrac,'cannabis.prod')
df_infrac = get_latest_data(dfcurrent,df_infrac,'cannabis.ventes')


#On effectue un merge les deux tables
df_infrac_legal = df_infrac.join(legal, on = 'Country')

df_infrac_legal

,Country,cannabis.conso_x,cannabis.conso_y,cannabis.prod,cannabis.ventes,Recreational,Medical
0,Austria,19909.0,19909.0,2601.0,6784.0,Decriminalized,Legal
1,Belgium,29572.0,29572.0,661.0,5805.0,Decriminalized,Legal
3,Croatia,5159.0,5159.0,252.0,1016.0,Decriminalized,Legal
4,Cyprus,563.0,563.0,18.0,166.0,Illegal,Legal
5,Czechia,6443.0,6443.0,184.0,728.0,Decriminalized,Legal
10,Germany,174876.0,174876.0,494.0,39366.0,Legal,Legal
11,Greece,5764.0,5764.0,537.0,3601.0,Illegal,Legal
12,Hungary,3437.0,3437.0,117.0,587.0,Illegal,Illegal
19,Netherlands,2413.0,2413.0,2245.0,2245.0,Decriminalized,Legal
22,Portugal,7020.0,7020.0,247.0,6112.0,Decriminalized,Legal


On voit déjà apparaître ici notre principal problème: à force d'écarter les lignes qui comportent des valeurs manquantes pour une seule des variables que l'on veut inclure, on se retrouve avec un jeu de données très restreint. C'est une situation particulièrement difficile puisque nous ne pouvons pas y faire grand chose, cela tient uniquement à l'absence de données fournies par l'EUDA pour beaucoup de pays.

On tentera tout de même d'entraîner notre modèle, mais en s'attendant à des résultats tout au plus médiocres.

**NOTE**: contrairement à notre premier entraînement du modèle, il nous est ici nécessaire de normaliser nos données pour chaque variable car les ordres de grandeurs peuvent varier d'une infraction à l'autre. Cela a donc été implémenté dans le code ci-dessous.

In [ ]:
from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# Recodage des variables catégorielles

df_infrac_legal['Recreational_num'] = df_infrac_legal['Recreational'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})
df_infrac_legal['Medical_num'] = df_infrac_legal['Medical'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})

X = df_infrac_legal[['cannabis.conso_x','cannabis.conso_y','cannabis.prod','cannabis.ventes']]
Y = df_infrac_legal['Recreational_num']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=3)

X_train = preprocessing.normalize(X_train)
X_test = preprocessing.normalize(X_test)

svm_classifier = svm.SVC(C=1, kernel='linear')

svm_classifier.fit(X_train, Y_train)

Y_pred = svm_classifier.predict(X_test)

accuracy_score = sklearn.metrics.accuracy_score(Y_test, Y_pred)

print('Précision du modèle: ', accuracy_score)
print("Taille de l'échantillon de test: ", len(Y_test))

Précision du modèle:  0.6666666666666666
Taille de l'échantillon de test:  3


On se retrouve le plus souvent avec un score de précision aux alentour de 0.66, ce qui est donc en soi un peu meilleur qu'une simple prédiction aléatoire. Cependant, il nous est impossible de tirer quoique ce soit de ce score puisque rappelons qu'il est calculé ainsi:

$Accuracy = \dfrac{VP + VN}{FP + FN + VP + VN}$

Avec VP les vrais positifs, FP les faux positifs, etc.
Autrement dit, le nombre de prédictions correctes divisées par la population totale de l'échantillon de test. Ici puisque la taille de l'échantillon de test est seulement de 3 observations, il est en réalité difficile d'inférer quoique ce soit de ce ratio. La taille de l'échantillon est donc définitivement trop faible pour que l'on puisse tirer des conclusions satisfaisantes de notre modèle.

### 3.3 Conclusions générales tirés de cet échec

On se rend donc finalement compte d'un problème général et plutôt flagrant qui frappe le jeu de données choisi: ce dernier comporte un nombre beaucoup trop faible d'observations pour tenter de faire de l'apprentissage supervisé comme nous aurions aimé le faire.

La nature même de notre sujet, qui se limite à 29 pays membres de l'Union Européenne, rend donc bien difficile la mise en place de modèles de machine learning afin de produire des prédictions satisfaisantes et des analyses pertinentes. 

Des pistes à explorer si l'on voulait tout de même essayer de prédire l'état de la législation dans un pays donné serait alors éventuellement de ne tenter de prédire que la législation d'un seul pays, en connaissant celle des autres pays qui présenteraient des caractéristiques similaires au pays dont on essaierait de prédire la législation. 

Par manque de temps et de moyens, ce n'est pas le chemin que nous avons choisi de suivre. Fort des enseignements tirés de cet échec, nous nous sommes plutôt tournés vers la mise en place de modèle qui semblent plus adaptés au jeu de données mis à notre disposition. En effet, étant donné que nous possédons un nombre très limités d'individus (les pays), mais d'énormément de variables et d'observation pour chacun de ses individus, il nous paraît alors plus pertinent de nous tourner vers des modèles de régression linaires, qui seront plus propice à prédire et expliquer d'autres variables que la législation. Nous avons donc tenté, après cela, de nous tourner vers une prédiction de la consommation.

A TERMINER UNE FOIS LE PREMIER MODELE MIS EN PLACE

## Regression logistique sur les même variables juste pour essayer de voir si c'est mieux

In [90]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=16)

# fit the model with data
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)

accuracy = sklearn.metrics.accuracy_score(Y_pred,Y_test)

print(accuracy)
#accuracy

0.6666666666666666
